In [2]:
!pip install paramiko

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   --------- ------------------------------ 0.8/3.2 MB 6.7 MB/s eta 0:00:01
   ------------------------------------ --- 2.9/3.2 MB 8.0 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 7.8 MB/s eta 0:00:00


In [10]:
import paramiko
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization
import os
import hashlib
import random
import string
import time

# Generate a secure RSA key pair for key exchange
private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
public_key = private_key.public_key()

def serialize_public_key(public_key):
    """Serialize the RSA public key for sharing."""
    return public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

def encrypt_key_with_rsa(aes_key, public_key):
    """Encrypt AES key using the RSA public key."""
    return public_key.encrypt(
        aes_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

def decrypt_key_with_rsa(encrypted_key, private_key):
    """Decrypt AES key using the RSA private key."""
    return private_key.decrypt(
        encrypted_key,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

def generate_key(password: str, salt: bytes):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(password.encode())

def encrypt_file(file_path: str, key: bytes):
    with open(file_path, 'rb') as f:
        plaintext = f.read()
    iv = os.urandom(16)  # Random Initialization Vector
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()

    encrypted_file_path = file_path + '.enc'
    with open(encrypted_file_path, 'wb') as f:
        f.write(iv + ciphertext)
    print(f"Encrypted file created: {encrypted_file_path}")
    return encrypted_file_path

def decrypt_file(file_path: str, key: bytes):
    with open(file_path, 'rb') as f:
        data = f.read()
    iv, ciphertext = data[:16], data[16:]
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()

    decrypted_file_path = file_path.replace('.enc', '')
    with open(decrypted_file_path, 'wb') as f:
        f.write(plaintext)
    return decrypted_file_path

def calculate_hash(file_path):
    """Calculate the SHA-256 hash of a file."""
    with open(file_path, 'rb') as f:
        file_hash = hashlib.sha256(f.read()).hexdigest()
    return file_hash

def create_test_file(file_path, size_in_kb):
    """Create a text file with random content of the specified size in KB."""
    content = ''.join(random.choices(string.ascii_letters + string.digits, k=1024))  # 1 KB of data
    with open(file_path, 'w') as f:
        for _ in range(size_in_kb):
            f.write(content)
    print(f"Test file created: {file_path} ({size_in_kb} KB)")

def sftp_transfer(host: str, port: int, username: str, password: str, local_path: str, remote_path: str):
    try:
        if not os.path.exists(local_path):
            raise FileNotFoundError(f"Local file does not exist: {local_path}")

        transport = paramiko.Transport((host, port))
        transport.connect(username=username, password=password)
        sftp = paramiko.SFTPClient.from_transport(transport)

        start_time = time.time()
        # Transfer file
        sftp.put(local_path, remote_path)
        end_time = time.time()

        print(f"File successfully transferred to {remote_path} on the server.")
        transfer_time = end_time - start_time
        print(f"Transfer time: {transfer_time:.4f} seconds")

        sftp.close()
        transport.close()
        return transfer_time
    except Exception as e:
        print(f"Error during SFTP transfer: {e}")
        return None

if __name__ == "__main__":
    # Configuration
    SFTP_HOST = "localhost"
    SFTP_PORT = 22
    SFTP_USERNAME = "sftpuser"
    SFTP_PASSWORD = "password123"
    REMOTE_FILE_PATH = "example.enc"  # Saves to the user's home directory

    # File sizes to test in MB
    file_sizes_mb = [10, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1024]

    # Create test files and measure performance
    for size in file_sizes_mb:
        file_name = f"example_{size}mb.txt"
        size_in_kb = size * 1024

        # Create test file
        create_test_file(file_name, size_in_kb)

        # Generate encryption key
        password = "securepassword"
        salt = os.urandom(16)
        aes_key = generate_key(password, salt)

        # Encrypt the AES key with RSA public key
        encrypted_aes_key = encrypt_key_with_rsa(aes_key, public_key)

        # Decrypt the AES key on the receiving side (simulation for this test)
        decrypted_aes_key = decrypt_key_with_rsa(encrypted_aes_key, private_key)

        print(f"Processing file: {file_name}")

        # Encrypt the file
        encrypted_file_path = encrypt_file(file_name, decrypted_aes_key)

        # Transfer encrypted file via SFTP
        transfer_time = sftp_transfer(SFTP_HOST, SFTP_PORT, SFTP_USERNAME, SFTP_PASSWORD, encrypted_file_path, REMOTE_FILE_PATH)

        # Output transfer time
        if transfer_time is not None:
            print(f"Transfer time for {file_name}: {transfer_time:.4f} seconds")

        # Decrypt the file (for testing)
        decrypted_file_path = decrypt_file(encrypted_file_path, decrypted_aes_key)

        # Verify file integrity
        original_hash = calculate_hash(file_name)
        decrypted_hash = calculate_hash(decrypted_file_path)
        if original_hash == decrypted_hash:
            print(f"File integrity verified for {file_name}.")
        else:
            print(f"File integrity check failed for {file_name}.")


Test file created: example_10mb.txt (10240 KB)
Processing file: example_10mb.txt
Encrypted file created: example_10mb.txt.enc
File successfully transferred to example.enc on the server.
Transfer time: 0.1432 seconds
Transfer time for example_10mb.txt: 0.1432 seconds
File integrity verified for example_10mb.txt.
Test file created: example_100mb.txt (102400 KB)
Processing file: example_100mb.txt
Encrypted file created: example_100mb.txt.enc
File successfully transferred to example.enc on the server.
Transfer time: 0.7945 seconds
Transfer time for example_100mb.txt: 0.7945 seconds
File integrity verified for example_100mb.txt.
Test file created: example_200mb.txt (204800 KB)
Processing file: example_200mb.txt
Encrypted file created: example_200mb.txt.enc
File successfully transferred to example.enc on the server.
Transfer time: 1.4709 seconds
Transfer time for example_200mb.txt: 1.4709 seconds
File integrity verified for example_200mb.txt.
Test file created: example_300mb.txt (307200 KB)
